## Pipeline SVM: Preprocesamiento, Búsqueda de Hiperparámetros y Evaluación

En las siguientes celdas se carga el dataset limpio, se define un pipeline que incluye preprocesamiento (escalado) y un clasificador SVM, se realiza búsqueda sistemática de hiperparámetros con `GridSearchCV`, y se evalúa el mejor modelo en un set de prueba. Se guardan resultados resumidos para extraer conclusiones.

In [ ]:
# SVM pipeline: carga datos, preprocesado, GridSearch y evaluación
import json
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:


# Rutas
DATA_PATH = Path('..') / 'data' / 'diabetes_012_cleaned.csv'
DICT_PATH = Path('..') / 'data' / 'dictionary.json'
OUT_DIR = Path('.') / 'svm_results'
OUT_DIR.mkdir(exist_ok=True)

print("=" * 70)
print("PIPELINE SVM: BÚSQUEDA DE HIPERPARÁMETROS CON VALIDACIÓN CRUZADA (10-FOLD)")
print("=" * 70)

# Cargar datos
print("\n[1/5] Cargando datos...")
df = pd.read_csv(DATA_PATH)
with open(DICT_PATH, 'r', encoding='utf-8') as f:
    data_dict = json.load(f)
print(f"✓ Datos cargados: {df.shape[0]} muestras, {df.shape[1]} columnas")

# Preparar X, y (suponemos columna objetivo 'Diabetes')
if 'Diabetes' not in df.columns:
    raise RuntimeError("No se encontró la columna 'Diabetes' en el CSV limpiado.")

X = df.drop(columns=['Diabetes'])
y = df['Diabetes']

# Determinar variables por tipo usando dictionary.json
ordinal_vars = [k for k, v in data_dict.items() if v.get('type') == 'ordinal' and k in X.columns]
categorical_vars = [k for k, v in data_dict.items() if v.get('type') == 'categorical' and k in X.columns]
numerical_vars = [k for k, v in data_dict.items() if v.get('type') == 'numerical' and k in X.columns]

# Fallback: si listas vacías, inferir por dtype
if not numerical_vars and not categorical_vars and not ordinal_vars:
    numerical_vars = X.select_dtypes(include=['int64','float64']).columns.tolist()
    categorical_vars = X.select_dtypes(include=['object','category']).columns.tolist()

print(f"\n[2/5] Análisis de variables:")
print(f"  - Numéricas: {len(numerical_vars)} ({', '.join(numerical_vars[:3])}...)" if numerical_vars else "  - Numéricas: 0")
print(f"  - Categóricas: {len(categorical_vars)} ({', '.join(categorical_vars[:3])}...)" if categorical_vars else "  - Categóricas: 0")
print(f"  - Ordinales: {len(ordinal_vars)} ({', '.join(ordinal_vars[:3])}...)" if ordinal_vars else "  - Ordinales: 0")

# Para el pipeline trataremos ordinal y numerical como numéricos
num_features = numerical_vars + ordinal_vars
cat_features = categorical_vars

# ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
], remainder='drop')

# Pipeline con SVM
pipe = Pipeline([
    ('preproc', preprocessor),
    ('clf', SVC(probability=True))
])

# Train-test split
print("\n[3/5] Dividiendo datos (80% train, 20% test)...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"✓ Train: {X_train.shape[0]} muestras | Test: {X_test.shape[0]} muestras")
print(f"  Distribución clase train: {dict(pd.Series(y_train).value_counts().sort_index())}")

# GridSearch de hiperparámetros (sistema básico, ajustar según tiempo disponible)
param_grid = {
    'clf__C': [0.1, 1, 10, 100],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma': ['scale', 'auto']
}

n_combinations = len(param_grid['clf__C']) * len(param_grid['clf__kernel']) * len(param_grid['clf__gamma'])
print(f"\n[4/5] Búsqueda de hiperparámetros con GridSearchCV (5-fold CV)...")
print(f"✓ Combinaciones a probar: {n_combinations}")
print(f"✓ Total de iteraciones: {n_combinations * 5} (combinaciones × folds)")

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)

# Ejecutar búsqueda (puede tardar)
print("\n⏳ Entrenando... (puede tardar varios minutos)\n")
grid.fit(X_train, y_train)

# Resultados
best = grid.best_estimator_
print('\n' + "=" * 70)
print("RESULTADOS DE VALIDACIÓN CRUZADA")
print("=" * 70)
print(f"\n✓ Mejores hiperparámetros encontrados:")
for param, value in grid.best_params_.items():
    print(f"  - {param}: {value}")
print(f"\n✓ Mejor score CV (f1_macro): {grid.best_score_:.4f}")

# Guardar CV results
cv_results = pd.DataFrame(grid.cv_results_)
cv_results.to_csv(OUT_DIR / 'gridcv_results.csv', index=False)

# Evaluación en test set
print("\n[5/5] Evaluando modelo en conjunto de prueba...")
y_pred = grid.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
print("\n" + "=" * 70)
print("MÉTRICAS EN CONJUNTO DE PRUEBA (TEST SET)")
print("=" * 70)
print(f"\n✓ Accuracy: {acc:.4f}")
print(f"✓ F1-Score (macro): {f1:.4f}")
print(f"\nReporte detallado por clase:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Test Set (SVM)')
plt.tight_layout()
plt.savefig(OUT_DIR / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Guardar resumen de métricas
summary = {
    'best_params': grid.best_params_,
    'cv_best_score_f1_macro': float(grid.best_score_),
    'test_accuracy': float(acc),
    'test_f1_macro': float(f1)
}
with open(OUT_DIR / 'summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("\n" + "=" * 70)
print("✅ EJECUCIÓN COMPLETADA")
print("=" * 70)
print(f"Resultados guardados en: {OUT_DIR.resolve()}")
print(f"  - gridcv_results.csv: Todos los resultados de validación cruzada")
print(f"  - confusion_matrix.png: Matriz de confusión")
print(f"  - summary.json: Resumen de métricas principales")

PIPELINE SVM: BÚSQUEDA DE HIPERPARÁMETROS CON VALIDACIÓN CRUZADA (10-FOLD)

[1/5] Cargando datos...
✓ Datos cargados: 253680 muestras, 22 columnas

[2/5] Análisis de variables:
  - Numéricas: 1 (BMI...)
  - Categóricas: 14 (HighBP, HighChol, CholCheck...)
  - Ordinales: 6 (GeneralHealth, MentalHealth, PhysicalHealth...)

[3/5] Dividiendo datos (80% train, 20% test)...
✓ Train: 202944 muestras | Test: 50736 muestras
  Distribución clase train: {0: np.int64(170962), 1: np.int64(3705), 2: np.int64(28277)}

[4/5] Búsqueda de hiperparámetros con GridSearchCV (5-fold CV)...
✓ Combinaciones a probar: 16
✓ Total de iteraciones: 80 (combinaciones × folds)

⏳ Entrenando... (puede tardar varios minutos)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

Detailed tracebacks of the workers should have been printed to stderr in the executor process if faulthandler was not disabled.